In [3]:
# Load necessary library and module
import warnings
warnings.filterwarnings('ignore')

import sys
import pandas as pd
import seaborn as sns
import numpy as np
import re
import string
import unicodedata
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from textblob import TextBlob, Word

[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/gitpod/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gitpod/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Apply text pre-processing to DataFrame
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
snowball = nltk.stem.SnowballStemmer('english')
porter_stemmer = PorterStemmer()

from textblob import TextBlob, Word

In [6]:
# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# stemming process
sentence = 'Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan'
output   = stemmer.stem(sentence)

In [8]:
# stemming(sentence)
print(output)

ekonomi indonesia sedang dalam tumbuh yang bangga


## LOAD DATASET

In [10]:
# dataset= pd.read_csv('dataset/DatasetPilpres2024_Twitter_10112023.csv', encoding='ISO-8859-1')
samples= pd.read_csv('hasil/samples.csv', encoding='ISO-8859-1')

In [67]:
dataset.describe()

,ï»¿id,likes,quotes,replies,retweets,user/totalFollowers,user/totalFollowing,user/totalLikes,user/totalTweets
count,5.830000e+03,5830.000000,5830.000000,5830.000000,5830.000000,5.830000e+03,5830.000000,5830.000000,5.830000e+03
mean,1.718972e+18,156.454374,4.195540,35.033276,77.761578,2.809121e+05,1576.078559,16623.094511,9.833891e+04
std,8.023115e+15,1085.247184,36.977125,387.327978,582.525123,1.466068e+06,6913.578377,62187.983826,3.647013e+05
min,1.607710e+18,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.000000e+00
25%,1.720430e+18,0.000000,0.000000,0.000000,0.000000,2.000000e+01,26.000000,55.000000,2.820000e+02
50%,1.721740e+18,0.000000,0.000000,0.000000,0.000000,1.900000e+02,102.000000,285.000000,2.100000e+03
75%,1.721840e+18,3.000000,0.000000,1.000000,1.000000,6.814000e+03,825.000000,3286.000000,2.689975e+04
max,1.721860e+18,30892.000000,1079.000000,7544.000000,16926.000000,1.105555e+07,191331.000000,828485.000000,2.411624e+06


In [69]:
# bank = bank[['content', 'score']]
# samples=dataset[['tanggal','text']].iloc[1:10]
samples=dataset[['tanggal','text','likes','replies','quotes','retweets','searchQuery']]

In [71]:
samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5830 entries, 0 to 5829
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tanggal      5830 non-null   object
 1   text         5830 non-null   object
 2   likes        5830 non-null   int64 
 3   replies      5830 non-null   int64 
 4   quotes       5830 non-null   int64 
 5   retweets     5830 non-null   int64 
 6   searchQuery  5830 non-null   object
dtypes: int64(4), object(3)
memory usage: 319.0+ KB


In [72]:
#lowercase all text
samples['text_lower']=(dataset['text'].str.lower())

In [73]:
print(samples.iloc[10])

tanggal                                               10/24/2023
text           Bakal calon presiden dan wakil presiden Koalis...
likes                                                       2924
replies                                                      647
quotes                                                       287
retweets                                                     441
searchQuery                                             #cakimin
text_lower     bakal calon presiden dan wakil presiden koalis...
Name: 10, dtype: object


In [74]:
samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5830 entries, 0 to 5829
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tanggal      5830 non-null   object
 1   text         5830 non-null   object
 2   likes        5830 non-null   int64 
 3   replies      5830 non-null   int64 
 4   quotes       5830 non-null   int64 
 5   retweets     5830 non-null   int64 
 6   searchQuery  5830 non-null   object
 7   text_lower   5830 non-null   object
dtypes: int64(4), object(4)
memory usage: 364.5+ KB


In [ ]:
# print(stemming(samples.text[1]))

In [53]:
samples.head()

,tanggal,text,clean_content
1,11/7/2023,result togel 07 november 2023 jogjatoto\n\n??s...,result togel 07 november 2023 jogjatoto sunda ...
2,11/7/2023,result togel 07 november 2023 jogjatoto\n\n??b...,result togel 07 november 2023 jogjatoto belaru...
3,11/7/2023,result togel 07 november 2023 jogjatoto\n\n??t...,result togel 07 november 2023 jogjatoto toto m...
4,11/7/2023,#politik #muhaiminiskandar #cakimin #mahasiswa...,politik muhaiminiskandar cakimin mahasiswaunis...
5,11/7/2023,result togel 07 november 2023 jogjatoto\n\n??j...,result togel 07 november 2023 jogjatoto japan ...


## STOPWORD REMOVAL

In [11]:
samplex= pd.read_csv('hasil/samples.csv', encoding='ISO-8859-1')

In [12]:
from nltk.corpus import stopwords

stopwords_id = stopwords.words('indonesian')
len(stopwords_id)

758

In [13]:
# Buat fungsi untuk langkah stopwords removal

def remove_stopwords(text):
  clean_word = []
  all_text = text.split()
  for word in all_text:
    if word not in stopwords_id:
      clean_word.append(word)
  return ' '.join(clean_word)

In [14]:
samples['clean_content'] = samplex['filtering'].progress_apply(lambda x: remove_stopwords(x))

  0%|          | 0/5830 [00:00<?, ?it/s]

## Normalisasi Kata

In [10]:
# Download corpus singkatan
# !pip install wget
# import wget
# wget.download('https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv','dataset/normalisasi.csv')

'dataset/normalisasi (1).csv'

In [75]:
key_norm = pd.read_csv('dataset/normalisasi.csv')

# Buat fungsi untuk melakukan word normalization
def normalisasi(text):
  text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])
  text = str.lower(text)
  return text

In [76]:
samples['norm_content'] = samples['text_lower'].progress_apply(lambda x: normalisasi(x))

  0%|          | 0/5830 [00:00<?, ?it/s]

In [77]:
def stemming(text):
    text = stemmer.stem(text)
    return text

In [78]:
samples['stem_content'] = samples['norm_content'].progress_apply(lambda x: stemming(x))

  0%|          | 0/5830 [00:00<?, ?it/s]

In [79]:
samples.head()

,tanggal,text,likes,replies,quotes,retweets,searchQuery,text_lower,norm_content,stem_content
0,11/7/2023,RESULT TOGEL 07 NOVEMBER 2023 JOGJATOTO\n\n??P...,0,0,0,0,#gibran,result togel 07 november 2023 jogjatoto\n\n??p...,result togel 07 november 2023 jogjatoto ??pale...,result togel 07 november 2023 jogjatoto palemb...
1,11/7/2023,RESULT TOGEL 07 NOVEMBER 2023 JOGJATOTO\n\n??S...,0,0,0,0,#gibran,result togel 07 november 2023 jogjatoto\n\n??s...,result togel 07 november 2023 jogjatoto ??sund...,result togel 07 november 2023 jogjatoto sunda ...
2,11/7/2023,RESULT TOGEL 07 NOVEMBER 2023 JOGJATOTO\n\n??B...,0,0,0,0,#gibran,result togel 07 november 2023 jogjatoto\n\n??b...,result togel 07 november 2023 jogjatoto ??bela...,result togel 07 november 2023 jogjatoto belaru...
3,11/7/2023,RESULT TOGEL 07 NOVEMBER 2023 JOGJATOTO\n\n??T...,0,0,0,0,#gibran,result togel 07 november 2023 jogjatoto\n\n??t...,result togel 07 november 2023 jogjatoto ??toto...,result togel 07 november 2023 jogjatoto toto m...
4,11/7/2023,#Politik #MuhaiminIskandar #cakimin #mahasiswa...,0,0,0,0,#cakimin,#politik #muhaiminiskandar #cakimin #mahasiswa...,#politik #muhaiminiskandar #cakimin #mahasiswa...,politik muhaiminiskandar cakimin mahasiswaunis...


In [62]:
samples.describe()

,tanggal,text,clean_content,norm_content
count,9,9,9,9
unique,2,9,9,9
top,11/7/2023,result togel 07 november 2023 jogjatoto\n\n??s...,result togel 07 november 2023 jogjatoto sunda ...,result togel 07 november 2023 jogjatoto sunda ...
freq,6,1,1,1


In [58]:
print(samples)

     tanggal                                               text  \
1  11/7/2023  result togel 07 november 2023 jogjatoto\n\n??s...   
2  11/7/2023  result togel 07 november 2023 jogjatoto\n\n??b...   
3  11/7/2023  result togel 07 november 2023 jogjatoto\n\n??t...   
4  11/7/2023  #politik #muhaiminiskandar #cakimin #mahasiswa...   
5  11/7/2023  result togel 07 november 2023 jogjatoto\n\n??j...   
6  11/6/2023  bacapres anies baswedan belum menunjuk ketua t...   
7  11/7/2023  hanya ganjar yang konsisten tolak israel datan...   
8  11/6/2023  nakama ga matten da! jama su na! ??? [sebuah u...   
9  11/6/2023  bertemu din syamsuddin, cak imin klaim menang ...   

                                       clean_content  \
1  result togel 07 november 2023 jogjatoto sunda ...   
2  result togel 07 november 2023 jogjatoto belaru...   
3  result togel 07 november 2023 jogjatoto toto m...   
4  politik muhaiminiskandar cakimin mahasiswaunis...   
5  result togel 07 november 2023 jogjatoto japan 

## FILTERING

In [80]:
def filtering(text):
  text = text.lower()                                   # Lowercase all sentences
      #   text = text.strip()                                   # Remove whitespace
  text = re.sub('[-+]?[0-9]+', ' ', text)               # Remove numbers 
  text = re.sub(r'https?://\S+|www\.\S+', ' ', text)    # Remove URLs
  text = re.sub(r"pic.twitter.com\S+", ' ', text)       # Remove custom URLs for twitter
  text = re.sub(r'\@([\w]+)',' ', text)                 # Remove Mention @
  text = re.sub(r'\#([\w]+)',' ', text)                 # Remove #TAGAR
  text = re.sub('\S*@\S*\s?', ' ', text)                # Remove email
  text = re.sub(r'[^\w\s]', ' ', text)                  # Remove punctuation
      #   text = re.sub(r'\b\w{1,3}\b','',text)                 #Remove n-chars,Remove less than 3 chars, minimum 4 character allowed "\b[a-zA-Z0-9]{3}\b"
  text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text)  # Tahap-5: simbol
      #   text = re.sub(r'[0-9]+','', text)                     # Tahap-6: angka
  text = re.sub(r'([a-zA-Z])\1\1','\\1', text)          # Tahap-7: koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
  text = re.sub(' +',' ', text)                         #remove multiple whitespace
  text = re.sub(r'^[ ]|[ ]$','', text)                  # Tahap-9: spasi di awal dan akhir kalimat

  # text = re.sub('\b[a-zA-Z0-9]{3}\b','',text)
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') # Remove non-ascii character
  return text

In [81]:
samples['filtering'] = samples['stem_content'].progress_apply(lambda x: filtering(x))


  0%|          | 0/5830 [00:00<?, ?it/s]

In [64]:
# samples['clean_stemming'] = samples['filtering'].progress_apply(lambda x: stemming(x))

  0%|          | 0/9 [00:00<?, ?it/s]

In [65]:
samples.to_csv( "hasil/samples_clean_content.csv", index=False, encoding='utf-8-sig')


In [ ]:
samples.to_csv( "hasil/samples.csv", index=False, encoding='utf-8-sig')

In [ ]:
def text_preprocessing(text):
  text = text.lower()                                   # Lowercase all sentences
      #   text = text.strip()                                   # Remove whitespace
  text = re.sub('[-+]?[0-9]+', ' ', text)               # Remove numbers 
  text = re.sub(r'https?://\S+|www\.\S+', ' ', text)    # Remove URLs
  text = re.sub(r"pic.twitter.com\S+", ' ', text)       # Remove custom URLs for twitter
  text = re.sub(r'\@([\w]+)',' ', text)                 # Remove Mention @
  text = re.sub(r'\#([\w]+)',' ', text)                 # Remove #TAGAR
  text = re.sub('\S*@\S*\s?', ' ', text)                # Remove email
  text = re.sub(r'[^\w\s]', ' ', text)                  # Remove punctuation
      #   text = re.sub(r'\b\w{1,3}\b','',text)                 #Remove n-chars,Remove less than 3 chars, minimum 4 character allowed "\b[a-zA-Z0-9]{3}\b"
  text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text)  # Tahap-5: simbol
      #   text = re.sub(r'[0-9]+','', text)                     # Tahap-6: angka
  text = re.sub(r'([a-zA-Z])\1\1','\\1', text)          # Tahap-7: koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
  text = re.sub(' +',' ', text)                         #remove multiple whitespace
  text = re.sub(r'^[ ]|[ ]$','', text)                  # Tahap-9: spasi di awal dan akhir kalimat

  # text = re.sub('\b[a-zA-Z0-9]{3}\b','',text)
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') # Remove non-ascii character
  word_tokens = word_tokenize(text) # Word tokenize
  

  # Define Indonesian stopwords removal
  stop_words = stopwords.words('english')  # NLTK Indonesian stopwords
  clean_words = [word for word in word_tokens if word not in stop_words] # stopwords removal
  clean_words = ' '.join(clean_words)

  words = set(nltk.corpus.words.words())
  word_cleaner = word_tokenize(clean_words)
  cleaner_words = [w for w in word_cleaner if w.lower() in words or not w.isalpha()] #remove non english
  cleaner = ' '.join(cleaner_words)

  #stemming with snowball update 07072022 stemmer terakhir
#   stem_token=word_tokenize(cleaner)
#   stem_words = [snowball.stem(w) for w in stem_token]
#   stem_clean= ' '.join(stem_words)

  #update13072022 pake lemmatization TextBlob
  tblob=TextBlob(clean_words)
  lemma= [Word(word).lemmatize("v") for word in tblob.words]
  lemma_clean= ' '.join(lemma)

  #   cleaner = " ".join(w for w in nltk.wordpunct_tokenize(clean_words) \
        #   if w.lower() in words or not w.isalpha())

  return lemma_clean